# Example of using rpy2 to call the `cppRouting` functions in the R library

We have worked out the nuances needed to call various cppRouting -- including the `assign_traffic` function, within Python using the rpy2 package.

In [ ]:
import os
os.environ['R_HOME'] = r"C:\Program Files\R\R-4.3.3"

from pathlib import Path

import pandas as pd
import geopandas as gpd
import numpy as np
import pyproj

from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
from rpy2 import robjects
from rpy2.robjects.conversion import localconverter

# Convert pandas.DataFrames to R dataframes automatically.
pandas2ri.activate()

cpp_routing = importr("cppRouting")
r_cpp_parallel = importr("RcppParallel")

data_directory = Path(r"C:\Users\Marc.Meketon\OneDrive - MMC\Documents\OliverWyman\DOE_IntermodalRouting\cppRouting\data_readme")
roads = pd.read_csv(data_directory / 'roads.csv', dtype={'from': str, 'to': str, 'weight': float})
com = gpd.read_file(data_directory / "com_simplified_geom.shp")
ndcom = pd.read_csv(data_directory / "node_commune.csv", dtype={"com": str, "id_noeud": str, "POPULATION": float})
med = pd.read_csv(data_directory / "doctor.csv", dtype={'CODGEO':str, 'NB_D201': float, 'id_noeud': str, 'POPULATION': float})
maternity = pd.read_csv(data_directory / "maternity.csv", dtype={"CODGEO": str, "NB_D107": float})
coord = pd.read_csv(data_directory / 'coordinates.csv', dtype={'ID': str, 'X': float, 'Y': float})

### Head of road network data

The weight is travel time in minutes.  The example does not say so explitly, but seems implicit when looking how they use the A* and NBA algorithms.  They had a constant of 110/0.06 for 110 kph (maximum speed) multiplied by 1000 to get meters per hour, then divided by 60 to get meters per minute.

In [ ]:
roads.head()

### Head of coordinates data

In [ ]:
coord.head()

### Instantiate the graph

In [ ]:
#Instantiate a graph with coordinates
graph  = cpp_routing.makegraph(roads, directed = True, coords = coord)

Attributes of the graph

In [ ]:
print(f"Members of graph: {graph.names}") 
print(f"Members of graph$data: {graph.rx2('data').names}")     # in R, it is graph$data
print(f"First 5 distances: {graph.rx2('data').rx2('dist')[0:5]}")
print(f"Number of vertices: {graph.rx2('nbnode')}")            # in R, it is graph$nbnode
print(f"Vertices ids: {graph.rx2('dict').rx2('ref')}")         # in R, it is graph$dict$ref

num_vertices = graph.rx2('nbnode')[0]
print(f"Number of vertices: {num_vertices}")

vertice_ids = graph.rx2('dict').rx2('ref')
print(f"Vertices ids: {vertice_ids}")

### Compare different path algorithms in terms of performance

Generate 2000 random origin and destination

In [ ]:
rng = np.random.default_rng()
origin = rng.choice(vertice_ids, 2000)
destination = rng.choice(vertice_ids, 2000)

In [ ]:
import time
import gc

def time_path_generation():
    gc.disable()

    t0 = time.perf_counter()
    pair_dijkstra = cpp_routing.get_distance_pair(graph, origin, destination, algorithm="Dijkstra")
    dijkstra_time = time.perf_counter() - t0

    t0 = time.perf_counter()
    pair_bidijkstra = cpp_routing.get_distance_pair(graph, origin, destination, algorithm="bi")
    bidijkstra_time = time.perf_counter() - t0

    t0 = time.perf_counter()
    pair_astar = cpp_routing.get_distance_pair(graph, origin, destination, algorithm="A*", constant=110.0/0.06)
    astar_time = time.perf_counter() - t0

    t0 = time.perf_counter()
    pair_nba = cpp_routing.get_distance_pair(graph, origin, destination, algorithm="NBA", constant=110.0/0.06)
    nba_time = time.perf_counter() - t0

    gc.enable()

    print(f'Dijkstra time: {dijkstra_time}')
    print(f'Bidirection Dijkstra time: {bidijkstra_time}')
    print(f'A* time: {astar_time}')
    print(f'NBA time: {nba_time}')

In [ ]:
original_num_threads = r_cpp_parallel.defaultNumThreads()[0]
print(f'Default num of threads used: {original_num_threads}')

r_cpp_parallel.setThreadOptions(numThreads=1)
print('Number of parallel threads set to 1')
time_path_generation()
print()

r_cpp_parallel.setThreadOptions(numThreads=4)
print('Number of parallel threads set to 4')
time_path_generation()
print()

r_cpp_parallel.setThreadOptions(numThreads=original_num_threads)
print(f'Number of parallel threads set to {original_num_threads}')
time_path_generation()

## Compute isochrones

An isochrone is a set of nodes reachable from a node within a fixed limit.
Let’s compute isochrones around Dijon city

In the example, they plot the isochrones.  Not done here, but see https://walker-data.com/posts/python-isochrones/ for an idea of how to do it in geopandas.

Another site:  https://geoffboeing.com/2016/11/osmnx-python-street-networks/

Note, I'm not sure how the computations are done.  In particular, the graph edges 'weights' (called 'dist' in the graph object) are discussed to be in minutes (not km).

The coordinates appear to be in the CRS 2154 (based on the R code found in the "compute isochrones" section of the example).  We can transform them to lat/longs using pyproj

In [ ]:
coordinate_transformer = pyproj.Transformer.from_crs("EPSG:2154", "EPSG:4326")
def get_x_y_from_vertice_id(vertice_id: int, coord_df: pd.DataFrame) -> tuple[float, float]:
    coordinate_transformer = pyproj.Transformer.from_crs("EPSG:2154", "EPSG:4326")
    x_y = coord_df[coord_df.ID == str(vertice_id)][["X", "Y"]].to_numpy()
    return coordinate_transformer.transform(*x_y[0])

In [ ]:
#  This appears to be the correct route between these two points:  When converted to lat/longs (below) and the route is calculated in Google Maps,
#    the intermediate points are in the path

path = cpp_routing.get_path_pair(graph, '205793', '88' )[0]
path

**The time between Dijon and point 88 is a bit baffling**:  the cell below suggests a travel time of 30 minutes.  Google says it is 48.7 km and 37 minutes (with no traffic).

In [ ]:
[(vertex_id, round(cpp_routing.get_distance_pair(graph, '205793', str(vertex_id))[0],2), get_x_y_from_vertice_id(vertex_id, coord) )
 for vertex_id in path]

The lat/long of 205793 does appear to be in the center of Dijon, France.

In [ ]:
iso  =  cpp_routing.get_isochrone(graph, "205793", lim = robjects.IntVector([15, 25, 45, 60, 90, 120]))

In [ ]:
print(f'iso.names: {iso.names[0]}')
print(f"iso.rx2('205793').names: {iso.rx2(str(iso.names[0])).names}")  # Note the change to a string
print(f"location that is 90 minutes out: {iso[0][4][0]} with X,Y coordinates: {coord[coord.ID==iso[0][4][0]]}")

In [ ]:
iso[0][4]

In [ ]:
iso[0][4][0]

In [ ]:
all(iso[0][0] == iso.rx2('205793').rx2('15'))  # all the vertices that are reachable within 15 minutes

In [ ]:
len(iso[0][0])

## Compute possible detours within a fixed additional cost

In [ ]:
#Compute shortest path
trajet = cpp_routing.get_path_pair(graph,"205793","212490")[0]
print(trajet)

#Compute shortest path
distance = cpp_routing.get_distance_pair(graph,"205793","212490")[0]
print(distance)

#Compute detour time of 25 and 45 minutes
det25 =cpp_routing.get_detour(graph,"205793","212490",extra=25)[0]
det45 = cpp_routing.get_detour(graph,"205793","212490",extra=45)[0]
print(len(det25), det25[0:10])
print(len(det45), det45[0:10])

## Compute Contraction Hierarchy

In [ ]:
graph3 = cpp_routing.cpp_contract(graph, silent=True)

In [ ]:

#Calculate distances on the contracted graph
t0 = time.perf_counter()
pair_ch = cpp_routing.get_distance_pair(graph3, origin, destination)
ch_time = time.perf_counter() - t0
print(f'Dijkstra using contraction hierarchy time: {ch_time}')

t0 = time.perf_counter()
pair_bidijkstra = cpp_routing.get_distance_pair(graph, origin, destination, algorithm="bi")
bidijkstra_time_time = time.perf_counter() - t0
print(f'BiDijkstra time: {bidijkstra_time_time}')

In [ ]:
pair_ch = np.array(pair_ch)
pair_bidijkstra = np.array(pair_bidijkstra)

When comparing the answers between the contraction hierarchy, leave out the situations in which there are no paths

In [ ]:
distances_when_paths_exist_ch = pair_ch[~np.isnan(pair_ch)]
distances_when_paths_exist_bidijkstra = pair_ch[~np.isnan(pair_bidijkstra)]

In [ ]:
np.sum(np.abs(np.array(distances_when_paths_exist_ch) - np.array(distances_when_paths_exist_bidijkstra)))

In [ ]:
len(distances_when_paths_exist_ch), len(distances_when_paths_exist_bidijkstra)

## Two parallel link example of user equilibrium

In the cppRouting documentation (https://github.com/vlarmet/cppRouting), it refers to an example in https://tfresource.org/topics/User_Equilibrium.html.  Below repeats the example.

Here we have two nodes, 1 and 2, and two parallel directed links going from node 1 to node 2.  We want to move 3000 units.  The first link has capacity of 2200 units, and can travel 60 mph.  The second link has capacity 1700 and travels at 45 mph.  The $\alpha$ and $\beta$ parameters are the same on both links and are $\alpha=0.15$ and $\beta=4.0$

If $x$ is the flow on link 1, then $3000-x$ is the flow on link 2.  We want to find the $x$ such that

$$\left[ \frac{60}{60} \right] \left( 1 + \alpha \left( \frac{x}{2200}\right)^ \beta \right) = \left[ \frac{60}{45} \right] * \left(1 + \alpha \left( \frac{3000-x}{1700}\right)^ \beta \right)$$

```python
from scipy.optimize import fsolve

f = lambda x: (1.0 + 0.15*((x/2200.0)**4.0)) - (60.0/45.0)*(1.0 + 0.15*(((3000.0-x)/1700.0)**4.0))
fsolve(f, 0)
```

returns
```array([2686.54933136])```

In [ ]:
two_link_df = pd.DataFrame({'From': [1, 1], 
                            'To': [2, 2], 
                            'Free_Flow_Time': [60.0/60.0, 60.0/45.0],
                            'Capacity': [2200.0, 1700.0],
                            'alpha': [0.15, 0.15],
                            'beta': [4.0, 4.0]})
two_link_df

Let's use Algorithm B from R. Dial, as implemented in `cppRouting`

In [ ]:
two_link_graph = cpp_routing.makegraph(two_link_df[['From', 'To', 'Free_Flow_Time']], 
                                       directed = True,
                                       capacity = two_link_df['Capacity'],
                                       alpha = two_link_df['alpha'],
                                       beta = two_link_df['beta'])

In [ ]:
# note:  we cannot use the 'from' argument because it is a Python keyword
#        we could either do one of the two lines below which rely on positional arguments

# flows = cpp_routing.assign_traffic(two_link_graph, [1], [2], demand=[3000], algorithm="dial")
# flows = cpp_routing.assign_traffic(two_link_graph, [1], to=[2], demand=[3000], algorithm="dial")

# or we can use argument-unpacking (see: https://stackoverflow.com/a/41902009/1955013)
flows = cpp_routing.assign_traffic(two_link_graph, demand=[3000], algorithm="dial", **{'from': [1], 'to': [2]})

In [ ]:
print(flows.names)
print(flows.rx2('gap'))
print(flows.rx2('iteration')[0])
print(flows.rx2('data').names)
print(flows.rx2('data'))
r_data_df = flows.rx2('data')
with localconverter(robjects.default_converter + pandas2ri.converter):
  pd_data_df = robjects.conversion.rpy2py(r_data_df)
pd_data_df

# Load in the Rail Network and see what simplify and contraction do

In [ ]:
narn_directory = Path(r'C:\Users\Marc.Meketon\OneDrive - MMC\Documents\OliverWyman\CN_IANR\inputs')
lines_gdf = gpd.read_file(narn_directory / 'North_American_Rail_Network_Lines.zip')

In [ ]:
lines_gdf.head(3)

In [ ]:
narn_lines_skinny_df = lines_gdf[["FRFRANODE", "TOFRANODE", "MILES"]]
narn_lines_skinny_df.rename(columns={"FRFRANODE": "from", "TOFRANODE": "to"})
narn_graph = cpp_routing.makegraph(narn_lines_skinny_df, directed=True)

In [ ]:
print(narn_graph.names)
print(narn_graph.rx2('data').names)
print(narn_graph.rx2('nbnode'))
print(narn_graph.rx2('data').rx2('from')[0:3])
print(narn_graph.rx2('data').rx2('to')[0:3])
print(narn_graph.rx2('data').rx2('dist')[0:3])
print(len(narn_graph.rx2('data').rx2('dist')))

In [ ]:
narn_graph_ch = cpp_routing.cpp_contract(narn_graph, silent=True)

In [ ]:
print(narn_graph_ch.names)
print(narn_graph_ch.rx2('data').names)
print(narn_graph_ch.rx2('nbnode'))
print(narn_graph_ch.rx2('data').rx2('from')[0:3])
print(narn_graph_ch.rx2('data').rx2('to')[0:3])
print(narn_graph_ch.rx2('data').rx2('dist')[0:3])
print(len(narn_graph_ch.rx2('data').rx2('dist')))

In [ ]:
narn_graph_simp = cpp_routing.cpp_simplify(narn_graph, rm_loop=False, iterate=True)

In [ ]:
print(narn_graph_simp.names)
print(narn_graph_simp.rx2('data').names)
print(narn_graph_simp.rx2('nbnode'))
print(narn_graph_simp.rx2('data').rx2('from')[0:3])
print(narn_graph_simp.rx2('data').rx2('to')[0:3])
print(narn_graph_simp.rx2('data').rx2('dist')[0:3])
print(len(narn_graph_simp.rx2('data').rx2('dist')))

In [ ]:
rng = np.random.default_rng()
vertice_ids_narn = narn_graph.rx2('dict').rx2('ref')
origin_narn = rng.choice(vertice_ids_narn, 2000)
destination_narn = rng.choice(vertice_ids_narn, 2000)
unique_narn_vertices = list(set(set(origin_narn) | set(destination_narn)))
len(unique_narn_vertices)

In [ ]:
narn_graph_simp = cpp_routing.cpp_simplify(narn_graph, rm_loop=False, iterate=True, keep=robjects.StrVector(unique_narn_vertices))

In [ ]:
t0 = time.perf_counter()
narn_pair = cpp_routing.get_distance_pair(narn_graph, origin_narn, destination_narn, algorithm = "bi")
narn_time = time.perf_counter() - t0
print(f'NARN time for 2000 OD pairs:  {narn_time}')

t0 = time.perf_counter()
narn_pair_ch = cpp_routing.get_distance_pair(narn_graph_ch, origin_narn, destination_narn, algorithm="phast")
narn_time_ch = time.perf_counter() - t0
print(f'NARN CH time for 2000 OD pairs:  {narn_time_ch}')

t0 = time.perf_counter()
narn_pair_simp = cpp_routing.get_distance_pair(narn_graph_simp, origin_narn, destination_narn, algorithm = "bi")
narn_time_simp = time.perf_counter() - t0
print(f'NARN SIMP time for 2000 OD pairs:  {narn_time_simp}')

In [ ]:
# waterway map (https://www.npms.phmsa.dot.gov/CNWData.aspx).  Commercially navigable
ww_gdf = gpd.read_file(r'C:\Users\Marc.Meketon\Downloads\CNW_V6_2024.zip')
ww_gdf.explore()

In [ ]:
# waterway map (https://geospatial-usace.opendata.arcgis.com/datasets/604cdc08fe7d43cb90a0584a0b198875_0/explore).  Mile marker
# NOTE:  the 'download shape file' did not work.  I downloaded the geoJSON and had QGIS export it a shapefile
mm_gdf = gpd.read_file(r'C:\Users\Marc.Meketon\Downloads\river_mile_markers.zip')
mm_gdf.explore()

In [ ]:
# waterway map (https://geodata.bts.gov/datasets/a8c39fdf822842f6836cd61986e9b5a5_0/explore).  Marine highway

mh_gdf = gpd.read_file(r'C:\Users\Marc.Meketon\Downloads\marine_highways.zip')
mh_gdf.explore()